In [19]:
from ast import mod
from turtle import mode
import gym
from copy import deepcopy
import os
import os.path as osp
import torch
from scipy import stats
from statistics import mean 
import numpy as np
from torch.optim import Adam
import itertools
import random
import torch.nn as nn
import argparse
def print_2f(*args):
    __builtins__.print(*("%.2f" % a if isinstance(a, float) else a
                         for a in args))

def get_action(o, md, deterministic=True):
    o = torch.as_tensor(o, dtype=torch.float32)
    a = md.act(o)
    return a

def save_state(env):
    return env.sim.get_state()

def restore_state(env, old_state):
    env.reset()
    env.sim.set_state(old_state)
    env.sim.forward()
    return env.get_obs()

# each 
def test_model(env, model, max_ep_len=None, num_episodes=20, interval = 1):
    o, r, d, ep_ret, ep_len, n = env.reset(), 0, False, 0, 0, 0
    total_rewards = []
    trajs = []
    mid_points = []
    while n < num_episodes:
        old_state = save_state(env)
        mid_points.append(old_state)
        if len(mid_points)-1 != ep_len:
            print("bug:", len(mid_points)-1, ep_len)
        a = get_action(o, model)
        o, r, d, _ = env.step(a)
        ep_ret += r
        ep_len += 1
        if d or (ep_len == max_ep_len):
            total_rewards.append(ep_ret)
            trajs.append(mid_points)
            mid_points = []
            o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0
            n += 1
    return total_rewards, trajs

def vmean(v):
    return sum(v) / len(v)

def get_good_trajs(env, model, num, max_ep_len=1000):
    total_rewards = []
    trajs = []
    old_r = []
    while len(total_rewards) < num:
        r, t = test_model(env, model, max_ep_len, num - len(total_rewards))
        old_r = old_r + r
        thr = vmean(old_r)
        for i in range(len(r)):
            if thr <= r[i] and len(t[i])>=500:
                total_rewards.append(r[i])
                trajs.append(t[i])
    return total_rewards, trajs

def run_extra_steps(env, o, ep_len, md, max_ep_len, step_num = 50):
    # return 0
    # print(env.done)
    total_r = 0
    for i in range(step_num):
        a = get_action(o, md)
        o, r, d, _ = env.step(a)
        total_r += r
        ep_len += 1
        if d or (ep_len == max_ep_len):
            # print(i, d, r, ep_len, max_ep_len)
            return (d, total_r)
    return (d, total_r)

def get_models(path, env_name, name):
    env = gym.make(env_name)
    fpath = osp.join(path, name)
    # print(fpath)
    models = []
    while 1:
        fname = osp.join(fpath, name + "_s" + str(len(models)) ,'pyt_save', 'model.pt')
        print(fname)
        if(osp.exists(fname)):
            model = torch.load(fname)
            models.append(model)
        else:
            break
    return models

In [20]:
path = '/home/lclan/spinningup/data/'
env = gym.make('Walker2d-v3')
models =  get_models(path, 'Walker2d-v3', 'Walker2d-v3_sac_base')
print(len(models))


/home/lclan/spinningup/data/Walker2d-v3_sac_base/Walker2d-v3_sac_base_s0/pyt_save/model.pt
/home/lclan/spinningup/data/Walker2d-v3_sac_base/Walker2d-v3_sac_base_s1/pyt_save/model.pt
/home/lclan/spinningup/data/Walker2d-v3_sac_base/Walker2d-v3_sac_base_s2/pyt_save/model.pt
/home/lclan/spinningup/data/Walker2d-v3_sac_base/Walker2d-v3_sac_base_s3/pyt_save/model.pt
/home/lclan/spinningup/data/Walker2d-v3_sac_base/Walker2d-v3_sac_base_s4/pyt_save/model.pt
4


In [21]:
r, t = get_good_trajs(env, models[0], 10)

In [24]:
print(t[0][0])

MjSimState(time=0.0, qpos=array([-4.59271761e-03,  1.25157847e+00,  1.93322821e-03,  4.18327324e-03,
        2.44796472e-04,  4.75745784e-03,  2.32210117e-03,  1.23040515e-03,
       -1.39541179e-03]), qvel=array([ 0.00476675, -0.00357611, -0.00270208,  0.00350662,  0.00401923,
       -0.00487908,  0.00446344, -0.00321702,  0.00463771]), act=None, udd_state={})


In [27]:
import pickle
with open('tj.pk', 'wb') as f:
    pickle.dump(t, f)

In [35]:
with open('tj.pk', 'rb') as f:
    t2 = pickle.load(f)


In [40]:
with open('./data/Walker2d-v3_sac_base_trajs.pkl', 'rb') as f:
    t2 = pickle.load(f)

In [42]:
len(t2[0])

1000

In [43]:
print(t2[0][0])

MjSimState(time=0.0, qpos=array([ 1.30687669e-03,  1.24915459e+00, -2.99255940e-03,  3.08362928e-03,
        2.44562962e-03,  8.19931050e-04, -2.47122719e-03, -1.82589574e-03,
        1.73980502e-03]), qvel=array([ 0.00100676, -0.00470481,  0.00306903, -0.00431403, -0.00309992,
       -0.00446594, -0.00267597,  0.00185501,  0.00275699]), act=None, udd_state={})
